In [1]:
# all codes come are changed based on LDR code.
# 用添加噪声的数据进行训练
import argparse
import torch.optim as optim
import os
import yaml
from mcrgan.default import _C as config
from mcrgan.default import update_config
import torch
from mcrgan.trainer import MCRTrainer
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.models import get_BNN_models
from mcrgan.models import get_noise
from mcrgan.models import get_noise_uniform
from mcrgan.models import DiscriminatorMNIST
from mcrgan.models import weights_init_mnist_model_noise

import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os
from mcrgan.loss import MCRGANloss
import random
import time
random.seed(0)

In [2]:
# 导入Fmnist数据集，进行对比
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET = 'FMNIST'
ROOT    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 8
DATA_SAMPLE = 50000
LABEL = 0

dataloader, dataset = get_dataloader(
        data_name=DATASET,
        root=ROOT,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)

In [3]:
import collections

def get_netG(netD,netV,net_noise,radius=1):
    models = [netD, netV,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    fed_state_dict = collections.OrderedDict()
    for key in weight_keys:
        fed_state_dict[key] = worker_state_dict[0][key] + radius*worker_state_dict[2][key]*torch.log(1+torch.exp(worker_state_dict[1][key]))
    return fed_state_dict

# new netG already optimized by adam and time learning rate, so we only using netG to update netV
def update_netV(netV,netG,net_noise,fed_state_dict,radius=1):
    models =[netV,netG,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[0][key] - (worker_state_dict[1][key]-fed_state_dict[key])*(worker_state_dict[2][key]*radius)/torch.log(1+torch.exp(-worker_state_dict[0][key]))
    return update_dict

# generate different distribution via gaussian init and uniform
def get_cauchy_noise(netD,net_noise1,net_noise2):
    models =[netD,net_noise1, net_noise2]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[1][key] / worker_state_dict[2][key]
    return update_dict
        

In [4]:
#models
# train hyperparameters, batch norm not used var
#store_Var = []
import time
from torch_mimicry.training import scheduler, logger, metric_log

from test_acc import nearsub
import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features2
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os

start_time = time.time()
#global_step = 0
epsilon_noise = 0.1
num_steps = 1000
n_dis = 1
n_dis2 = 1
numclasses =10
lr1 = 1e-3
lr2 = 1e-3
log_data = metric_log.MetricLog()
data_time = time.time()
iter_dataloader = iter(dataloader)
data, label = next(iter_dataloader)
data_time = time.time() - data_time
real_cpu = data.to(device)
real_label = label.clone().detach()

netD, netG, netV = get_BNN_models(DATASET, device, LABEL)
net_noise = get_noise(DATASET, device, LABEL)
fed_state_dict = get_netG(netD,netV,net_noise)
netG.load_state_dict(fed_state_dict)
optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))

Acc_results_D = []
Acc_results_G = []


transform = transforms.Compose(
            [transforms.Resize(32),
             transforms.ToTensor(),
             transforms.Normalize(0.5, 0.5)])

trainset = datasets.FashionMNIST(root=ROOT, train=True, download=True, transform=transform)
testset = datasets.FashionMNIST(root=ROOT, train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(
        testset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)

trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
n_comp = 10


In [5]:
# golden search algorithm to determined the radius of the sample ball, due to traditional updating netV is too slow
# no meaning for n_dis =2 for our case, just to have output for every update
# sample too much would arise cuda memory error
def gs_radius(radius1,radius2,netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2


def gs_radius2(radius1,radius2, netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise(DATASET, device, LABEL)
    net_noise1 = get_noise(DATASET, device, LABEL)
    net_noise_cauchy = get_cauchy_noise(netD,net_noise,net_noise1)
    net_noise.load_state_dict(net_noise_cauchy)
    #net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2

def gs_radius3(radius1,radius2, netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise_uniform(DATASET, device, LABEL)

    #net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2

In [6]:
#acc at first for NetD
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(testloader, netD, netG)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netD, netG)


print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

#acc at first for NetD
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetD, Acc at first',count/10000)
Acc_results_D.append(count/10000)

#acc at first for NetG
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)


#acc at first
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetG, Acc at first',count/10000)
Acc_results_G.append(count/10000)

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.95s/it]


----------------------------
Train_z, Test_z
PCA: 0.8318
SVD: 0.8241
NetD, Acc at first 0.8397


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.68s/it]


----------------------------
Train_z, Test_z
PCA: 0.8071
SVD: 0.8084
NetG, Acc at first 0.8439


In [7]:
# data selected
mean_V = []
n_dis2 = 1
n_dis = 1
mode = 11
gam3 = 1
mcr_gan_loss = MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
loss_V = 0
last_V = -1000
num_steps = 500
lr1 = 1e-3
lr2 = 1e-3
optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
meanV_sum = []
radius_list = []
radius = 5

for epoch in range(num_steps):
    if 1==1:
        
        #radius,_ = gs_radius(radius1=0,radius2=100,netD=netD,real_cpu=real_cpu,real_label=real_label,tor=1e-5,n_dis=2,lam=0.618,mode=mode)
        #radius_list.append(radius)
        #print(radius)

        for i in range(n_dis):
            netD.zero_grad()
            optD.zero_grad()
            Z = netD(real_cpu)
            net_noise = get_noise(DATASET, device, LABEL)
            #radius,_ = gs_radius(radius1=0,radius2=100,netD=netD,real_cpu=real_cpu,real_label=real_label,tor=1e-5,n_dis=2,lam=0.618,mode=mode)
            #radius_list.append(radius)
            #print(radius)
            fed_state_dict = get_netG(netD,netV,net_noise,radius)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errD, errD_EC = mcr_gan_loss(Z, Z_bar, real_label, 1, n_dis)
            print(errD, errD_EC)
            errD.backward()
            optD.step()
        temp = 0
        for j in range(n_dis2):
            netG.zero_grad()
            optG.zero_grad()

            Z = netD(real_cpu)
            net_noise = get_noise(DATASET, device, LABEL)
            radius,_ = gs_radius(radius1=0,radius2=100,netD=netD,real_cpu=real_cpu,real_label=real_label,tor=1e-5,n_dis=2,lam=0.618,mode=mode)
            radius_list.append(radius)
            print(radius)
            fed_state_dict = get_netG(netD,netV,net_noise,radius)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG, errG_EC = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            print(errG, errG_EC)
            temp = errG
            errG = (-1) * errG
            errG.backward()
            optG.step()

            update_dic = update_netV(netV,netG,net_noise,fed_state_dict,radius)
            netV.load_state_dict(update_dic)

        loss_V = temp
        

tensor(-16.2486, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-5.9605, device='cuda:0', grad_fn=<NegBackward0>), tensor(-6.5231, device='cuda:0', grad_fn=<NegBackward0>), tensor(-42.1640, device='cuda:0', grad_fn=<AddBackward0>)]
7.631828604437706
tensor(-13.9489, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-3.8175, device='cuda:0', grad_fn=<NegBackward0>), tensor(-6.4748, device='cuda:0', grad_fn=<NegBackward0>), tensor(-37.7282, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-13.4207, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-3.8175, device='cuda:0', grad_fn=<NegBackward0>), tensor(-5.9846, device='cuda:0', grad_fn=<NegBackward0>), tensor(-36.2852, device='cuda:0', grad_fn=<AddBackward0>)]
99.30339162743611
tensor(-17.9997, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-8.2007, device='cuda:0', grad_fn=<NegBackward0>), tensor(-5.9384, device='cuda:0', grad_fn=<NegBackward0>), tensor(-46.4914, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-19.3134, device='cud

0.4709984279915666
tensor(-49.1393, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-23.8841, device='cuda:0', grad_fn=<NegBackward0>), tensor(-23.2021, device='cuda:0', grad_fn=<NegBackward0>), tensor(-6.7922, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-48.9599, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-23.8841, device='cuda:0', grad_fn=<NegBackward0>), tensor(-22.9814, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.1205, device='cuda:0', grad_fn=<AddBackward0>)]
0.5187234173325131
tensor(-49.4810, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-24.0497, device='cuda:0', grad_fn=<NegBackward0>), tensor(-23.3561, device='cuda:0', grad_fn=<NegBackward0>), tensor(-6.9666, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-49.4088, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-24.0497, device='cuda:0', grad_fn=<NegBackward0>), tensor(-23.2413, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.3123, device='cuda:0', grad_fn=<AddBackward0>)]
0.4226336039628021
ten

0.4545021175348197
tensor(-59.3330, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-29.2403, device='cuda:0', grad_fn=<NegBackward0>), tensor(-28.3314, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.8201, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-59.4019, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-29.2403, device='cuda:0', grad_fn=<NegBackward0>), tensor(-28.2971, device='cuda:0', grad_fn=<NegBackward0>), tensor(-5.4527, device='cuda:0', grad_fn=<AddBackward0>)]
0.4699362895824925
tensor(-59.8212, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-29.3289, device='cuda:0', grad_fn=<NegBackward0>), tensor(-28.6132, device='cuda:0', grad_fn=<NegBackward0>), tensor(-5.5477, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-59.7959, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-29.3289, device='cuda:0', grad_fn=<NegBackward0>), tensor(-28.5831, device='cuda:0', grad_fn=<NegBackward0>), tensor(-5.5791, device='cuda:0', grad_fn=<AddBackward0>)]
0.4516141480871292
ten

0.36256915815758994
tensor(-66.5327, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-32.8749, device='cuda:0', grad_fn=<NegBackward0>), tensor(-32.1728, device='cuda:0', grad_fn=<NegBackward0>), tensor(-3.4146, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-66.5614, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-32.8749, device='cuda:0', grad_fn=<NegBackward0>), tensor(-32.2379, device='cuda:0', grad_fn=<NegBackward0>), tensor(-3.2569, device='cuda:0', grad_fn=<AddBackward0>)]
0.39563920098397165
tensor(-66.5403, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-32.8607, device='cuda:0', grad_fn=<NegBackward0>), tensor(-32.1640, device='cuda:0', grad_fn=<NegBackward0>), tensor(-3.5520, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-66.4614, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-32.8607, device='cuda:0', grad_fn=<NegBackward0>), tensor(-32.0428, device='cuda:0', grad_fn=<NegBackward0>), tensor(-3.7486, device='cuda:0', grad_fn=<AddBackward0>)]
0.4005445366520736
t

0.35804688666235607
tensor(-70.2735, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-34.8547, device='cuda:0', grad_fn=<NegBackward0>), tensor(-34.0961, device='cuda:0', grad_fn=<NegBackward0>), tensor(-2.7537, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-70.2929, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-34.8547, device='cuda:0', grad_fn=<NegBackward0>), tensor(-33.9978, device='cuda:0', grad_fn=<NegBackward0>), tensor(-3.2225, device='cuda:0', grad_fn=<AddBackward0>)]
0.34560747628517147
tensor(-70.6501, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-35.0053, device='cuda:0', grad_fn=<NegBackward0>), tensor(-34.3439, device='cuda:0', grad_fn=<NegBackward0>), tensor(-2.6729, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-70.7207, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-35.0053, device='cuda:0', grad_fn=<NegBackward0>), tensor(-34.4571, device='cuda:0', grad_fn=<NegBackward0>), tensor(-2.5197, device='cuda:0', grad_fn=<AddBackward0>)]
0.30946312996025027


...

0.24963447344217252
tensor(-95.2809, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.5675, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.1612, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.7370, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-95.2935, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.5675, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.1260, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.8221, device='cuda:0', grad_fn=<AddBackward0>)]
0.25971396865252816
tensor(-95.4175, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.6772, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.1182, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.8630, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-95.4697, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.6772, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.1416, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.9174, device='cuda:0', grad_fn=<AddBackward0>)]
0.2386542949138326
t

0.24484222597739225
tensor(-95.4239, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.6734, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.2113, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.7147, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-95.3851, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.6734, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.1363, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.7778, device='cuda:0', grad_fn=<AddBackward0>)]
0.26053251269836064
tensor(-95.5629, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.7499, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.1668, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.9082, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-95.6129, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.7499, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.2618, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.8243, device='cuda:0', grad_fn=<AddBackward0>)]
0.2301376846862292
t

0.2515173817299767
tensor(-95.9678, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.9442, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.4714, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.7371, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-95.9833, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.9442, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.4966, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.7204, device='cuda:0', grad_fn=<AddBackward0>)]
0.2708516934948494
tensor(-95.9851, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.9737, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.3549, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.9279, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-96.0187, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.9737, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.3846, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.9354, device='cuda:0', grad_fn=<AddBackward0>)]
0.2564186407552599
ten

0.2390462768962657
tensor(-96.1643, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.0635, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.5411, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.7501, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-96.1780, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.0635, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.6144, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.6489, device='cuda:0', grad_fn=<AddBackward0>)]
0.2421944968575352
tensor(-96.3782, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.1538, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.6710, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.7391, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-96.4756, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.1538, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.8279, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.6387, device='cuda:0', grad_fn=<AddBackward0>)]
0.212047009493638
tens

0.23709526550253873
tensor(-96.2931, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.1005, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.6989, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.6384, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-96.3114, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.1005, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.7101, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.6502, device='cuda:0', grad_fn=<AddBackward0>)]
0.21763589487036902
tensor(-96.4724, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.1528, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.8743, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.5610, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-96.3772, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.1528, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.7751, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.5672, device='cuda:0', grad_fn=<AddBackward0>)]
0.2405233622676396
t

In [8]:
radius_list

[7.631828604437706,
 99.30339162743611,
 1.162022820119659,
 1.0734194102794905,
 0.7935370145225645,
 0.7076883254037893,
 0.7248378336424347,
 0.5851525595132528,
 0.6221818096160284,
 0.46453989034545434,
 0.6347552207199472,
 0.5697286346394675,
 0.5351230692891956,
 0.5545086434142497,
 0.5553099280569015,
 0.4569709389468453,
 0.5786298828360196,
 0.4709984279915666,
 0.5187234173325131,
 0.4226336039628021,
 0.4614879531348951,
 0.4523501803891282,
 0.4897876010642077,
 0.4556062165748179,
 0.48226948262444147,
 0.4539753491262107,
 0.46198862574137856,
 0.4649946850891548,
 0.37988847961474376,
 0.4263794877634226,
 0.37656793197660715,
 0.4916613364356101,
 0.45790124422450623,
 0.45489557453770385,
 0.4545021175348197,
 0.4699362895824925,
 0.4516141480871292,
 0.42776682948953926,
 0.4553738952556769,
 0.4184625442244911,
 0.43893030239368497,
 0.4057320008702529,
 0.42598014296283504,
 0.3918799154527211,
 0.39079331087960656,
 0.4145957914702556,
 0.39805817042443514,
 0.3

In [9]:
#acc at first for NetD
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(testloader, netD, netG)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netD, netG)


print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

#acc at first for NetD
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetD, Acc at first',count/10000)
Acc_results_D.append(count/10000)

#acc at first for NetG
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)


#acc at first
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetG, Acc at first',count/10000)
Acc_results_G.append(count/10000)


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.73s/it]


----------------------------
Train_z, Test_z
PCA: 0.841
SVD: 0.8356
NetD, Acc at first 0.8609


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


----------------------------
Train_z, Test_z
PCA: 0.8311999999999999
SVD: 0.8309
NetG, Acc at first 0.8589


In [10]:
torch.save(netD, 'FMNIST_NetD_changing1-mode11.pth')
torch.save(netV, 'FMNIST_NetV_changing1-mode11.pth')
torch.save(netG, 'FMNIST_NetG_changing1-mode11.pth')

#netD = torch.load('FMNIST_NetD.pth')
#netV = torch.load('FMNIST_NetV.pth')
#netG = torch.load('FMNIST_NetG.pth')

#mode = 1
#gam3 = 1

In [11]:
r_list = [0,0.1,0.2,0.5,1,2,3,4,6,8,10]
sample_num = 20

In [12]:
# sample 20 times of NetG at final radius, NetD is fixed, length 0.1,0.2,0.5,1,2,4,6,8,10,15,20
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample = []
for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[1])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample.append(count/10000)
    
print(max(Acc_results_G_sample),min(Acc_results_G_sample),np.mean(Acc_results_G_sample),np.var(Acc_results_G_sample))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


----------------------------
Train_z, Test_z
PCA: 0.8327
SVD: 0.8301000000000001
NetG_fixD, Acc at sample 1 0.8586


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


----------------------------
Train_z, Test_z
PCA: 0.8328
SVD: 0.8291999999999999
NetG_fixD, Acc at sample 2 0.8593


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.8329
SVD: 0.8291999999999999
NetG_fixD, Acc at sample 18 0.8589


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


----------------------------
Train_z, Test_z
PCA: 0.833
SVD: 0.8294
NetG_fixD, Acc at sample 19 0.8587


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.67s/it]


----------------------------
Train_z, Test_z
PCA: 0.8325
SVD: 0.8288
NetG_fixD, Acc at sample 20 0.8598
0.8605 0.858 0.85892 4.5760000000000955e-07


In [13]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample2 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[2])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample2.append(count/10000)
    
print(max(Acc_results_G_sample2),min(Acc_results_G_sample2),np.mean(Acc_results_G_sample2),np.var(Acc_results_G_sample2))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


----------------------------
Train_z, Test_z
PCA: 0.8337
SVD: 0.8304
NetG_fixD, Acc at sample 1 0.8599


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


----------------------------
Train_z, Test_z
PCA: 0.8342
SVD: 0.8303
NetG_fixD, Acc at sample 2 0.8584


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.8323
SVD: 0.8295
NetG_fixD, Acc at sample 18 0.8585


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.62s/it]


----------------------------
Train_z, Test_z
PCA: 0.833
SVD: 0.8291
NetG_fixD, Acc at sample 19 0.8577


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


----------------------------
Train_z, Test_z
PCA: 0.8341000000000001
SVD: 0.8302
NetG_fixD, Acc at sample 20 0.8581
0.8599 0.8571 0.8582650000000001 3.942750000000103e-07


In [14]:
# sample 50 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample3 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[3])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample3.append(count/10000)
    
print(max(Acc_results_G_sample3),min(Acc_results_G_sample3),np.mean(Acc_results_G_sample3),np.var(Acc_results_G_sample3))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.67s/it]


----------------------------
Train_z, Test_z
PCA: 0.8326
SVD: 0.8301000000000001
NetG_fixD, Acc at sample 1 0.8568


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.69s/it]


----------------------------
Train_z, Test_z
PCA: 0.8296
SVD: 0.8269
NetG_fixD, Acc at sample 2 0.8569


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.8319
SVD: 0.8282
NetG_fixD, Acc at sample 18 0.8558


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.63s/it]


----------------------------
Train_z, Test_z
PCA: 0.8315
SVD: 0.8282
NetG_fixD, Acc at sample 19 0.8577


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


----------------------------
Train_z, Test_z
PCA: 0.8323
SVD: 0.8291999999999999
NetG_fixD, Acc at sample 20 0.8563
0.8581 0.8541 0.8568650000000002 8.752750000000034e-07


In [15]:
# sample 50 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample4 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[4])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample4.append(count/10000)
    
print(max(Acc_results_G_sample4),min(Acc_results_G_sample4),np.mean(Acc_results_G_sample4),np.var(Acc_results_G_sample4))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


----------------------------
Train_z, Test_z
PCA: 0.8325
SVD: 0.8281000000000001
NetG_fixD, Acc at sample 1 0.8547


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


----------------------------
Train_z, Test_z
PCA: 0.8299
SVD: 0.8269
NetG_fixD, Acc at sample 2 0.8546


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.8311
SVD: 0.8265
NetG_fixD, Acc at sample 18 0.8558


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


----------------------------
Train_z, Test_z
PCA: 0.8304
SVD: 0.8274
NetG_fixD, Acc at sample 19 0.8529


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


----------------------------
Train_z, Test_z
PCA: 0.829
SVD: 0.8293
NetG_fixD, Acc at sample 20 0.8553
0.8562 0.8523 0.8543499999999999 1.0964999999999869e-06


In [16]:
# sample 50 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample5 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[5])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample5.append(count/10000)
    
print(max(Acc_results_G_sample5),min(Acc_results_G_sample5),np.mean(Acc_results_G_sample5),np.var(Acc_results_G_sample5))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


----------------------------
Train_z, Test_z
PCA: 0.8293
SVD: 0.8214
NetG_fixD, Acc at sample 1 0.8419


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


----------------------------
Train_z, Test_z
PCA: 0.8273
SVD: 0.8271
NetG_fixD, Acc at sample 2 0.8392


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.8249
SVD: 0.8243
NetG_fixD, Acc at sample 18 0.8401


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


----------------------------
Train_z, Test_z
PCA: 0.8286
SVD: 0.8241
NetG_fixD, Acc at sample 19 0.8455


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


----------------------------
Train_z, Test_z
PCA: 0.8278
SVD: 0.8236
NetG_fixD, Acc at sample 20 0.8422
0.8471 0.8352 0.8424499999999998 8.485499999999992e-06


In [17]:
# sample 50 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample6 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[6])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample6.append(count/10000)
    
print(max(Acc_results_G_sample6),min(Acc_results_G_sample6),np.mean(Acc_results_G_sample6),np.var(Acc_results_G_sample6))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


----------------------------
Train_z, Test_z
PCA: 0.8168
SVD: 0.8135
NetG_fixD, Acc at sample 1 0.7774


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


----------------------------
Train_z, Test_z
PCA: 0.8248
SVD: 0.8184
NetG_fixD, Acc at sample 2 0.806


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.8216
SVD: 0.8153
NetG_fixD, Acc at sample 18 0.8134


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.63s/it]


----------------------------
Train_z, Test_z
PCA: 0.8214
SVD: 0.8140000000000001
NetG_fixD, Acc at sample 19 0.7964


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.63s/it]


----------------------------
Train_z, Test_z
PCA: 0.8214
SVD: 0.8166
NetG_fixD, Acc at sample 20 0.7684
0.8134 0.7308 0.784965 0.0003302172750000005


In [18]:
# sample 50 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample7 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[7])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample7.append(count/10000)
    
print(max(Acc_results_G_sample7),min(Acc_results_G_sample7),np.mean(Acc_results_G_sample7),np.var(Acc_results_G_sample7))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


----------------------------
Train_z, Test_z
PCA: 0.8086
SVD: 0.8066
NetG_fixD, Acc at sample 1 0.4874


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


----------------------------
Train_z, Test_z
PCA: 0.8082
SVD: 0.8063
NetG_fixD, Acc at sample 2 0.3119


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.8054
SVD: 0.8059000000000001
NetG_fixD, Acc at sample 18 0.4531


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


----------------------------
Train_z, Test_z
PCA: 0.8141
SVD: 0.8147
NetG_fixD, Acc at sample 19 0.5135


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


----------------------------
Train_z, Test_z
PCA: 0.8122
SVD: 0.8091
NetG_fixD, Acc at sample 20 0.4201
0.5391 0.267 0.44016500000000003 0.004976039274999999


In [19]:
# sample 50 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample8 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[8])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample8.append(count/10000)
    
print(max(Acc_results_G_sample8),min(Acc_results_G_sample8),np.mean(Acc_results_G_sample8),np.var(Acc_results_G_sample8))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


----------------------------
Train_z, Test_z
PCA: 0.8181
SVD: 0.8181
NetG_fixD, Acc at sample 1 0.1646


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


----------------------------
Train_z, Test_z
PCA: 0.8143
SVD: 0.8108
NetG_fixD, Acc at sample 2 0.121


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.8067
SVD: 0.8049
NetG_fixD, Acc at sample 18 0.1253


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


----------------------------
Train_z, Test_z
PCA: 0.8102
SVD: 0.8085
NetG_fixD, Acc at sample 19 0.12


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


----------------------------
Train_z, Test_z
PCA: 0.806
SVD: 0.8049
NetG_fixD, Acc at sample 20 0.1845
0.1845 0.1156 0.14316999999999996 0.0004090141


In [20]:
# sample 50 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample9 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[9])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample9.append(count/10000)
    
print(max(Acc_results_G_sample9),min(Acc_results_G_sample9),np.mean(Acc_results_G_sample9),np.var(Acc_results_G_sample9))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


----------------------------
Train_z, Test_z
PCA: 0.8044
SVD: 0.804
NetG_fixD, Acc at sample 1 0.0824


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


----------------------------
Train_z, Test_z
PCA: 0.8069999999999999
SVD: 0.8095
NetG_fixD, Acc at sample 2 0.1174


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.806
SVD: 0.8077
NetG_fixD, Acc at sample 18 0.1115


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.67s/it]


----------------------------
Train_z, Test_z
PCA: 0.8054
SVD: 0.8058
NetG_fixD, Acc at sample 19 0.0978


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.68s/it]


----------------------------
Train_z, Test_z
PCA: 0.8086
SVD: 0.8099000000000001
NetG_fixD, Acc at sample 20 0.0856
0.1409 0.074 0.10813999999999999 0.00026769840000000005


In [21]:
# sample 50 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample10 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[10])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample10.append(count/10000)
    
print(max(Acc_results_G_sample10),min(Acc_results_G_sample10),np.mean(Acc_results_G_sample10),np.var(Acc_results_G_sample10))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


----------------------------
Train_z, Test_z
PCA: 0.8023
SVD: 0.8038
NetG_fixD, Acc at sample 1 0.0745


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


----------------------------
Train_z, Test_z
PCA: 0.8126
SVD: 0.8046
NetG_fixD, Acc at sample 2 0.0736


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.8071
SVD: 0.8007
NetG_fixD, Acc at sample 18 0.0982


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.63s/it]


----------------------------
Train_z, Test_z
PCA: 0.8097
SVD: 0.8078
NetG_fixD, Acc at sample 19 0.1048


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


----------------------------
Train_z, Test_z
PCA: 0.8099000000000001
SVD: 0.8038
NetG_fixD, Acc at sample 20 0.1142
0.1245 0.0736 0.102525 0.00023918087499999997
